<a href="https://colab.research.google.com/github/LEE-spacexmark25/ai-crypto--/blob/master/phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving real.csv to real.csv


라이브러리 세팅

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_trx = pd.read_csv("real.csv")

헤더추가

In [ ]:
df_trx.columns = ["Price", "Volume", "BuySell(0/1)", "Timestamp"]

최종 Rawdata

In [ ]:
print(df_trx)

            Price  Volume  BuySell(0/1)            Timestamp
0        38802000  0.0514             0  2022-05-20 14:15:50
1        38801000  0.4240             0  2022-05-20 14:15:50
2        38800000  0.0003             0  2022-05-20 14:15:50
3        38798000  0.0542             0  2022-05-20 14:15:50
4        38821000  0.0323             1  2022-05-20 14:15:50
...           ...     ...           ...                  ...
1624714  38592000  0.5173             1  2022-05-22 14:47:49
1624715  38593000  0.2048             1  2022-05-22 14:47:49
1624716  38594000  0.2591             1  2022-05-22 14:47:49
1624717  38595000  0.4900             1  2022-05-22 14:47:49
1624718  38597000  0.0316             1  2022-05-22 14:47:49

[1624719 rows x 4 columns]


In [ ]:
##매도호가(Sell) 한 개가 비어있음. 추가
top_row = pd.DataFrame({'Price':[38803000],'Volume':[0.0427],'BuySell(0/1)':[0],'Timestamp':['2022-05-20 14:15:50']})
# Concat with old DataFrame and reset the Index.
df_trx = pd.concat([top_row, df_trx]).reset_index(drop = True)

15:15:50의 데이터가 몇 번째에 위치한지 알아보기

In [ ]:
print(df_trx[df_trx['Timestamp'] == '2022-05-20 15:15:50'].index.values)

[33350 33351 33352 33353 33354 33355 33356 33357 33358 33359]


33359번째까지의 데이터만 반환하도록 Dataframe Slicing

In [ ]:
df_trx = df_trx.iloc[:33360,:]

In [ ]:
print(df_trx)

          Price  Volume  BuySell(0/1)            Timestamp
0      38803000  0.0427             0  2022-05-20 14:15:50
1      38802000  0.0514             0  2022-05-20 14:15:50
2      38801000  0.4240             0  2022-05-20 14:15:50
3      38800000  0.0003             0  2022-05-20 14:15:50
4      38798000  0.0542             0  2022-05-20 14:15:50
...         ...     ...           ...                  ...
33355  37965000  0.2670             1  2022-05-20 15:15:50
33356  37967000  0.1472             1  2022-05-20 15:15:50
33357  37969000  0.3934             1  2022-05-20 15:15:50
33358  37971000  0.1406             1  2022-05-20 15:15:50
33359  37973000  0.0488             1  2022-05-20 15:15:50

[33360 rows x 4 columns]


같은 Timestamp들만 모아보기

In [ ]:
time_stamps = np.unique(df_trx['Timestamp'])

In [ ]:
print(time_stamps)

['2022-05-20 14:15:50' '2022-05-20 14:15:51' '2022-05-20 14:15:52' ...
 '2022-05-20 15:15:48' '2022-05-20 15:15:49' '2022-05-20 15:15:50']


Dataframe에서 같은 Timestamp의 data만 가져오기


*   예시로 14:15:52 인 것만 가져옴



In [ ]:
df_same = df_trx[df_trx['Timestamp'] == time_stamps[2]]

In [ ]:
print(df_same)

       Price  Volume  BuySell(0/1)            Timestamp
20  38815000  0.0010             0  2022-05-20 14:15:52
21  38807000  0.0526             0  2022-05-20 14:15:52
22  38803000  0.0870             0  2022-05-20 14:15:52
23  38802000  0.0514             0  2022-05-20 14:15:52
24  38801000  0.4240             0  2022-05-20 14:15:52
25  38821000  0.0323             1  2022-05-20 14:15:52
26  38822000  0.4036             1  2022-05-20 14:15:52
27  38824000  0.0553             1  2022-05-20 14:15:52
28  38825000  0.0537             1  2022-05-20 14:15:52
29  38826000  0.0317             1  2022-05-20 14:15:52


Midprice : 0의 최대가격과 1의 최소가격의 차이


1.   원래 정상적으로 데이터를 받아오면 한 Timestamp에 Buy 5개, Sell 5개 씩 총 10개의 데이터가 찍혀야 함. 
2.   그런데 혹시 몰라 10개를 다 못 받아오는 경우도 있을 것 같아서 별도로 0에서의 최대와 1에서의 최소를 찾아서 가져오는 과정을 거침



매수(0) 최대

In [ ]:
bid_max = df_same.loc[df_same['BuySell(0/1)'] == 0, 'Price'].max()

매도(1) 최소

In [ ]:
ask_min = df_same.loc[df_same['BuySell(0/1)'] == 1, 'Price'].min()

Midprice 구하기

In [ ]:
mid_price = (bid_max + ask_min) * 0.5

Midprice 결과

In [ ]:
print(mid_price)

37959000.0


Bookprice 구하기

1.   Bookprice란?

    *   midprice의 경우 최상단의 매수가격과 최하단의 매도가격만 보기 때문에 그 밖의       Level에서 일어나는 현상(그 밑에 가격에서 대량 매수/매도 주문을 넣는다던지)의 현상을 포착 불가

    *   그 현상을 해결하기 위해서 매수/매도의 양 5단계 가격과 주문대기량을 모두 반영할 수 있게끔 해줄 필요가 있음.


2.   How?

    *   각 매수/매도 level(각각 총 5단계로 이루어진)의 주문 대기량에 0.2제곱씩을 해준다음 모두 더해줘서 마치 주문 대기량의 지수평균을 낸 것 처럼 평균 매수/매도 주문 대기량을 산출

    *   각 매수/매도 level(각각 총 5단계로 이루어진)의 가격에 0.2제곱씩을 해준다음 모두 더해줘서 마치 주문 대기량의 지수평균을 낸 것 처럼 평균 매수/매도 가격을 산출

    *  이후 슬라이드 처럼 교차로 곱을 해주는 식을 적용하여 Bookprice 산출 









매도 5단계 수량 가져오기

In [ ]:
bid_volumes = df_same[df_same['BuySell(0/1)'] == 0]['Volume'].values
print(bid_volumes)

[0.0052 0.0304 0.0576 0.0524 0.0309]


매수 5단계 수량 가져오기

In [ ]:
ask_volumes = df_same[df_same['BuySell(0/1)'] == 1]['Volume'].values
print(ask_volumes)

[0.267  0.1472 0.3934 0.1406 0.0488]


가중평균(bidQty,askQty) 구하기

In [ ]:
import math

ask_qty_list = list()
for volume in ask_volumes:
  ask_qty_list.append(math.pow(volume, 0.2))

ask_qty = sum(ask_qty_list)
print(ask_qty)

bid_qty_list = list()
for volume in bid_volumes:
  bid_qty_list.append(math.pow(volume, 0.2))

bid_qty = sum(bid_qty_list)
print(bid_qty)



3.501438826693226
2.46492837817487


매수 5단계 가격 가져오기

In [ ]:
bid_prices = df_same[df_same['BuySell(0/1)'] == 0]['Price'].values
print(bid_prices)

[37953000 37949000 37947000 37946000 37941000]


매도 5단계 가격 가져오기

In [ ]:
ask_prices = df_same[df_same['BuySell(0/1)'] == 1]['Price'].values
print(ask_prices)

[37965000 37967000 37969000 37971000 37973000]


가중평균(bidPx, askPx) 구하기

In [ ]:
print(bid_qty_list)

[0.34930167541811463, 0.49724991404242036, 0.5650469000989533, 0.5544549402002757, 0.4988749484151062]


In [ ]:
print(ask_qty_list)

[0.7678957293584956, 0.681681584856692, 0.8297874671920346, 0.6754560004848926, 0.5466180448011114]


In [ ]:


bid_px = sum([a * b for a, b in zip(bid_prices, bid_qty_list)])
print(bid_px)

ask_px = sum([a * b for a, b in zip(ask_prices, ask_qty_list)])
print(ask_px)
'''
for price in sell_prices:
  ask_px += math.pow(price, 0.2)

print(ask_px)

bid_px = 0
for price in buy_prices:
  bid_px += math.pow(price, 0.2)

print(bid_px)
'''

93536179.7718517
132945233.24880813


'\nfor price in sell_prices:\n  ask_px += math.pow(price, 0.2)\n\nprint(ask_px)\n\nbid_px = 0\nfor price in buy_prices:\n  bid_px += math.pow(price, 0.2)\n\nprint(bid_px)\n'

Bookprice 구하기



In [ ]:
book_price = (((ask_qty*bid_px)/bid_qty) + ((bid_qty*ask_px)/ask_qty))/(bid_qty+ask_qty)
print(book_price)

37955874.216060676


Book imbalance 구하기(interval은 1)



In [ ]:
book_imbalance = book_price - mid_price
print(book_imbalance)

-3125.7839393243194


데이터 전체에 대한 Book_imbalance 구하기


*   앞선 예시에 for문만 씌워주면 됨



In [ ]:
#계산 결과를 담을 최종 Dataframe
df_result = pd.DataFrame(columns = ['Delta', 'Imbalance', 'midprice', 'Timestamp'])

#loop 밖에서 초기화
cur_ask_qty = 0
cur_bid_qty = 0

cur_ask_top = 0
cur_bid_top = 0

prev_ask_qty = 0
prev_bid_qty = 0

prev_ask_top = 0
prev_bid_top = 0

bid_side_add = 0
bid_side_count = 0
bid_side_delete = 0
bid_side_flip = 0
bid_side_trade = 0

ask_side_add = 0
ask_side_count = 0
ask_side_delete = 0
ask_side_flip = 0
ask_side_trade = 0
    ##loop 밖에서 초기화




for i in range(0, len(time_stamps)):
  current_time = time_stamps[i]
  df_same = df_trx[df_trx['Timestamp'] == current_time]

  bid_max = df_same.loc[df_same['BuySell(0/1)'] == 0, 'Price'].max()
  ask_min = df_same.loc[df_same['BuySell(0/1)'] == 1, 'Price'].min()
  mid_price = (ask_min + bid_max) * 0.5

  bid_volumes = df_same[df_same['BuySell(0/1)'] == 0]['Volume'].values
  ask_volumes = df_same[df_same['BuySell(0/1)'] == 1]['Volume'].values

  ask_qty_list = list()
  for volume in ask_volumes:
    ask_qty_list.append(math.pow(volume, 0.2))

  ask_qty = sum(ask_qty_list)


  bid_qty_list = list()
  for volume in bid_volumes:
    bid_qty_list.append(math.pow(volume, 0.2))

  bid_qty = sum(bid_qty_list)

  bid_prices = df_same[df_same['BuySell(0/1)'] == 0]['Price'].values
  ask_prices = df_same[df_same['BuySell(0/1)'] == 1]['Price'].values

  bid_px = sum([a * b for a, b in zip(bid_prices, bid_qty_list)])


  ask_px = sum([a * b for a, b in zip(ask_prices, ask_qty_list)])


  book_price = (((ask_qty*bid_px)/bid_qty) + ((bid_qty*ask_px)/ask_qty))/(bid_qty+ask_qty)
  book_imbalance = book_price - mid_price

  book_delta = 0

  if(i >= 1):

    cur_bid_qty = bid_qty
    cur_ask_qty = ask_qty
    cur_bid_top = bid_max
    cur_ask_top = ask_min

    if cur_bid_qty > prev_bid_qty:
      bid_side_add += 1
      bid_side_count += 1

    if cur_bid_qty < prev_bid_qty:
      bid_side_delete += 1
      bid_side_count += 1

    if cur_ask_qty > prev_ask_qty:
      ask_side_add += 1
      ask_side_count += 1

    if cur_ask_qty < prev_ask_qty:
      ask_side_delete += 1
      ask_side_count += 1

    if cur_bid_top < prev_bid_top:
      bid_side_flip += 1
      bid_side_count += 1

    if cur_ask_top > prev_ask_top:
      ask_side_flip += 1
      ask_side_count += 1

    ##len(0인것)
    bid_side_trade += 5
    bid_side_count += 5
    ##len(1인것)
    ask_side_trade += 5
    ask_side_count += 5

    bid_book_v = (-bid_side_delete + bid_side_add - bid_side_flip)/(math.pow(bid_side_count, 0.2))
    ask_book_v = (-ask_side_delete + ask_side_add - ask_side_flip)/(math.pow(ask_side_count, 0.2))
    trade_v = (ask_side_trade/math.pow(ask_side_count,0.2))-(bid_side_trade/math.pow(bid_side_count,0.2))

    book_delta = bid_book_v + ask_book_v + trade_v

    decay = math.exp(-1)

    bid_side_add *= decay
    bid_side_count *= decay
    bid_side_delete *= decay
    bid_side_flip *= decay
    bid_side_trade *= decay

    ask_side_add *= decay
    ask_side_count *= decay
    ask_side_delete *= decay
    ask_side_flip *= decay
    ask_side_trade *= decay

    prev_ask_qty = cur_ask_qty
    prev_bid_qty = cur_bid_qty

    prev_ask_top = cur_ask_top
    prev_bid_top = cur_bid_top

  new_row = {'Delta':book_delta, 'Imbalance':book_imbalance, 'midprice':mid_price, 'Timestamp':current_time}
  df_result = df_result.append(new_row, ignore_index=True)



최종 결과

In [ ]:
print(df_result)

         Delta    Imbalance    midprice            Timestamp
0            0 -1065.260552  38812000.0  2022-05-20 14:15:50
1     0.592746 -5731.375562  38818000.0  2022-05-20 14:15:51
2     0.208005 -4479.932592  38818000.0  2022-05-20 14:15:52
3    -0.986511 -1260.235712  38814000.0  2022-05-20 14:15:53
4     0.383257  -523.569277  38811500.0  2022-05-20 14:15:54
...        ...          ...         ...                  ...
3331 -1.184283 -2036.522517  37958000.0  2022-05-20 15:15:46
3332 -0.450157 -2036.522517  37958000.0  2022-05-20 15:15:47
3333 -0.385302 -3013.636629  37959000.0  2022-05-20 15:15:48
3334  0.373247 -2918.026958  37959000.0  2022-05-20 15:15:49
3335 -0.386088 -3125.783939  37959000.0  2022-05-20 15:15:50

[3336 rows x 4 columns]


Book delta 구하기

슬라이드 식 그대로 옮겼음

In [ ]:
#loop 밖에서 초기화
cur_ask_qty = 0
cur_bid_qty = 0

cur_ask_top = 0
cur_bid_top = 0

prev_ask_qty = 0
prev_bid_qty = 0

prev_ask_top = 0
prev_bid_top = 0

bid_side_add = 0
bid_side_count = 0
bid_side_delete = 0
bid_side_flip = 0
bid_side_trade = 0

ask_side_add = 0
ask_side_count = 0
ask_side_delete = 0
ask_side_flip = 0
ask_side_trade = 0
##loop 밖에서 초기화

if cur_bid_qty > prev_bid_qty:
  bid_side_add += 1
  bid_side_count += 1

if cur_bid_qty < prev_bid_qty:
  bid_side_delete += 1
  bid_side_count += 1

if cur_ask_qty > prev_ask_qty:
  ask_side_add += 1
  ask_side_count += 1

if cur_ask_qty < prev_ask_qty:
  ask_side_delete += 1
  ask_side_count += 1

if cur_bid_top < prev_bid_top:
  bid_side_flip += 1
  bid_side_count += 1

if cur_ask_top > prev_ask_top:
  ask_side_flip += 1
  ask_side_count += 1

##len(0인것)
bid_side_trade += 5
bid_side_count += 5
##len(1인것)
ask_side_trade += 5
ask_side_count += 5

bid_book_v = (-bid_side_delete + bid_side_add - bid_side_flip)/(math.pow(bid_side_count, 0.2))
ask_book_v = (-ask_side_delete + ask_side_add - ask_side_flip)/(math.pow(ask_side_count, 0.2))
trade_v = (ask_side_trade/math.pow(ask_side_count,0.2))-(bid_side_trade/math.pow(bid_side_count,0.2))

book_delta = bid_book_v + ask_book_v + trade_v

decay = math.exp(-1)

bid_side_add *= decay
bid_side_count *= decay
bid_side_delete *= decay
bid_side_flip *= decay
bid_side_trade *= decay

ask_side_add *= decay
ask_side_count *= decay
ask_side_delete *= decay
ask_side_flip *= decay
ask_side_trade *= decay

prev_ask_qty = cur_ask_qty
prev_bid_qty = cur_bid_qty

prev_ask_top = cur_ask_top
prev_bid_top = cur_bid_top



ZeroDivisionError: ignored